In [117]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

Hacer lista con los equipos por pais

In [118]:
LISTA_RONDAS = ["Final", "Final", "Semifinal", "Semifinal", "Cuartos", "Cuartos", "Octavos", "Octavos"]
LISTA_NUM_LINK = ["torneo=1048", "torneo=1047", "torneo=1046", "torneo=1045", 
                "torneo=1044", "torneo=1043", "torneo=1074", "torneo=1089", "libertadores"]
DICC_MES = {'Enero': '1', 'Febrero': '2', 'Marzo': '3', 'Abril': '4', 'Mayo': '5', 'Junio': '6', 'Julio': '7', 'Agosto': '8', 
            'Septiembre': '9', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'}
DICT_PAIS_EQUIPO = {'Dep.Tachira': 'venezuela', 'Dep.Tolima': 'colombia'}

In [119]:
def revisar_equipo(l, equipo, pais):
    cont = 0
    for i in l:
        eq = i.replace(",", "")
        eq = eq.replace(".", "")
        eq = eq.replace(" ", "")
        if eq[:3].upper() == equipo.upper():
            return i, pais[cont]
        cont += 1
    return "Dep. Cali", "colombia"

In [120]:
def imagen_a_pais(l):
    pais = []
    for i in l:
        imagen = i
        imagen = imagen.find('img')
        imagen = imagen.get('src')
        imagen = imagen.split("/")
        imagen = imagen[2].split(".")
        imagen = imagen[0]
        pais.append(imagen)
    return pais

In [121]:
locales = []
visitas = []
gol_locales = []
gol_visitas = []
resultados = []
goles = []
dif_goles = []
dias = []
anos = []
fases = []
paises_visita = []
paises_local = []
for num in LISTA_NUM_LINK:
    cont = 0
    url = f"https://www.promiedos.com.ar/{num}"
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_fases = sopa.find_all('div', id='partoff')
    final_unica = sopa.find_all('div', id='partoff1')

    if num == "libertadores":
        url = "https://www.promiedos.com.ar/libertadores=faseprevia"
        r = rq.get(url)
        sopa = bs(r.content, "html.parser")

    lista_grupos = sopa.find_all('div', class_ = 'grupo')
    for data in lista_grupos:
        equipos = data.find('table', id='posiciones')
        equipos = equipos.find_all('td', align = 'left')
        pais = imagen_a_pais(equipos)
        equipos = [i.text for i in equipos]
        partidos = data.find_all('div', id = 'fixgrupo')
        for datos in partidos:
            horario = datos.find_all('div', class_ = 'diahoragr')
            res = datos.find_all('div', class_ = 'grtr')

            for j in range(2):
                eq1 = res[j].find('div', class_ = 'greq1')
                eq1, p1 = revisar_equipo(equipos, eq1.text, pais)
                eq1 = eq1.replace(' ', '')
                locales.append(eq1)
                paises_local.append(p1)
                eq2 = res[j].find('div', class_ = 'greq2')
                eq2, p2 = revisar_equipo(equipos, eq2.text, pais)
                eq2 = eq2.replace(' ', '')
                visitas.append(eq2)
                paises_visita.append(p2)
                DICT_PAIS_EQUIPO[eq1] = p1
                DICT_PAIS_EQUIPO[eq2] = p2

                resultado = res[j].find('div', class_ = 'grres4')
                resultado = resultado.text
                resultado = resultado.split('-')
                res1 = int(resultado[0])
                res2 = int(resultado[1])
                gol_locales.append(res1)
                gol_visitas.append(res2)

                if res1 == res2:
                    resultados.append("Empate")
                    dif_goles.append(0)

                elif int(res1) > int(res2):
                    resultados.append("Local")
                    dif_goles.append(int(res1) - int(res2))

                elif int(res1) < int(res2):
                    resultados.append("Visita")
                    dif_goles.append(int(res2) - int(res1))

                goles.append(int(res1) + int(res2))
                anos.append(ano)
                dia = horario[j].text
                dia = dia.replace(",", "") ; dia = dia.split(" ")
                dia = dia[3]
                dias.append(dia)
                fases.append("Grupos")
    if final_unica != []:
        equ = final_unica[0].find_all('td', class_='game-t1')
        equ1 = equ[1].text.replace(' ', '') ; equ2 = equ[2].text.replace(' ', '')
        locales.append(equ1)
        visitas.append(equ2)
        paises_local.append(DICT_PAIS_EQUIPO[equ1])
        paises_visita.append(DICT_PAIS_EQUIPO[equ2])

        res1 = final_unica[0].find_all('td', class_='game-r1')
        res1 = (res1[1].text)[0]
        gol_locales.append(int(res1))
        res2 = final_unica[0].find_all('td', class_='game-r2')
        res2 = (res2[1].text)[0]
        gol_visitas.append(int(res2))

        if int(res1) == int(res2):
            resultados.append("Empate")
            dif_goles.append(0)

        elif int(res1) > int(res2):
            resultados.append("Local")
            dif_goles.append(int(res1) - int(res2))

        elif int(res1) < int(res2):
            resultados.append("Visita")
            dif_goles.append(int(res2) - int(res1))

        goles.append(int(res1) + int(res2))


        dia = final_unica[0].find_all('tr', class_='diapart')
        dia = dia[0].text
        dia = dia.split(" ")
        if len(dia) <= 4:
            dia.append("2022")
        anos.append(dia[4])
        ano = dia[4]
        dia = dia[1] + '/' + DICC_MES[dia[3]]
        dias.append(dia)
        fases.append("Final")
        cont = 2

    for data in lista_fases:
        lista_partidos = data.find_all('div', id='fixturein')
        fase = LISTA_RONDAS[cont]
        for datos in lista_partidos:
            equ = datos.find_all('td', class_='game-t1')
            equ1 = equ[1].text.replace(' ', '') ; equ2 = equ[2].text.replace(' ', '')
            locales.append(equ1)
            visitas.append(equ2)
            paises_local.append(DICT_PAIS_EQUIPO[equ1])
            paises_visita.append(DICT_PAIS_EQUIPO[equ2])

            res1 = datos.find_all('td', class_='game-r1')
            res1 = (res1[1].text)[0]
            gol_locales.append(int(res1))
            res2 = datos.find_all('td', class_='game-r2')
            res2 = (res2[1].text)[0]
            gol_visitas.append(int(res2))

            if int(res1) == int(res2):
                resultados.append("Empate")
                dif_goles.append(0)

            elif int(res1) > int(res2):
                resultados.append("Local")
                dif_goles.append(int(res1) - int(res2))

            elif int(res1) < int(res2):
                resultados.append("Visita")
                dif_goles.append(int(res2) - int(res1))

            goles.append(int(res1) + int(res2))


            dia = datos.find_all('tr', class_='diapart')
            dia = dia[0].text
            dia = dia.split(" ")
            if len(dia) <= 4:
                dia.append("2022")
            anos.append(dia[4])
            ano = dia[4]
            dia = dia[1] + '/' + DICC_MES[dia[3]]
            dias.append(dia)

            fases.append(fase)

        cont += 1
        if cont == 8:
            break

        

In [124]:
df_libertadores = pd.DataFrame()
df_libertadores["Local"] = locales
df_libertadores['Pais_local'] = paises_local
df_libertadores["Visita"] = visitas
df_libertadores['Pais_visita'] = paises_visita
df_libertadores["Gol_local"] = gol_locales
df_libertadores["Gol_visita"] = gol_visitas
df_libertadores["Resultado"] = resultados
df_libertadores["Año"] = anos
df_libertadores["Fase"] = fases
df_libertadores["Goles"] = goles
df_libertadores["Diferencia_goles"] = dif_goles
df_libertadores["Dia"] = dias


In [125]:
df_libertadores

,Local,Pais_local,Visita,Pais_visita,Gol_local,Gol_visita,Resultado,Año,Fase,Goles,Diferencia_goles,Dia
0,Universitario(P),peru,Velez,argentina,0,1,Visita,2022,Grupos,1,1,11/2
1,Paranaense,brasil,TheStrongest,bolivia,1,0,Local,2022,Grupos,1,1,13/2
2,TheStrongest,bolivia,Universitario(P),peru,1,0,Local,2022,Grupos,1,1,20/2
3,Velez,argentina,Paranaense,brasil,2,0,Local,2022,Grupos,2,2,25/2
4,TheStrongest,bolivia,Velez,argentina,2,0,Local,2022,Grupos,2,2,11/3
...,...,...,...,...,...,...,...,...,...,...,...,...
1111,Emelec,ecuador,Atl.Mineiro,brasil,1,1,Empate,2022,Octavos,2,0,28/6
1112,Dep.Tolima,colombia,Flamengo,brasil,0,1,Visita,2022,Octavos,1,1,29/6
1113,Corinthians,brasil,BocaJuniors,argentina,0,0,Empate,2022,Octavos,0,0,28/6
1114,Talleres(C),argentina,Colon,argentina,1,1,Empate,2022,Octavos,2,0,29/6
